In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import *
import dask.dataframe as dd
import pytz
import os
import gc
import glob
import re

pd.set_option('display.max_rows',9000)

In [2]:
###  UPLOAD MEDIROUTES DATA - format in csv

#up load all files containing Mediroutes data
dt = dd.read_csv('C:\\Users\\vjron\\OneDrive\\Desktop\\data science\\Rubicund Finance\\Mediroutes\\Mediroutes*.csv',dtype={'DL Number': 'float64','Vehicle': 'float64','Paid': 'float64','Distance Difference': 'float64','Notification Status': 'object','Patient Address': 'object','Dropoff Zip':'int64','Pickup Zip':'int64'}).compute()

# Rename and keep only the important columns
dt = dt.rename(columns={'Patient':'Name','Charge':'Cost','Import Distance':'Miles'})

#Assign Trip Legs to the trips
dt['Leg'] = 'A'

# Assign 'A' to the 'Duplicate' column for non-duplicate rows
dt.loc[~dt.duplicated(['Name','Date']), 'Leg'] = 'A'

# Assign 'B' to the 'Duplicate' column for duplicate rows
dt.loc[dt.duplicated(['Name','Date'],'first'), 'Leg'] = 'B'

print('Mediroutes data uploaded!!')

dt.to_csv('C:\\Users\\vjron\\OneDrive\\Desktop\\Rubicund Data.csv',index=False)


Mediroutes data uploaded!!


In [3]:
#### UPLOAD ALL ALIVI EOP'S - format in Excel .xlsx

# List all the Excel files in the folder
folder = 'C:\\Users\\vjron\\OneDrive\\Desktop\\data science\\Rubicund Finance\\Alivi'
filenames = os.listdir(folder)

# Create an empty list to store the DataFrames
df_list = []

# Loop through the files
for file in filenames:
    # Check if the file is an Excel file
    if file.endswith('.xlsx'):
        # Read the Excel file and skip the first 2 rows
        df = pd.read_excel((os.path.join(folder, file)),skiprows=[0, 1])
        # Add the DataFrame to the list
        df_list.append(df)

# Concatenate all the DataFrames into a single DataFrame
alv_all = pd.concat(df_list, ignore_index=True)

# Convert time column to string
alv_all['Scheduled Time'] = alv_all['Scheduled Time'].astype(str)

# Split the date time format to show only the date
alv_all['Date'] = alv_all['Scheduled Time'].apply(lambda x : x.split(' ')[0])

# Filter only needed columns, drop na and sort by date and name and reset index
alv_all = alv_all.loc[:,['Reference Number','Date','Member Name','Miles','Cost']].dropna().sort_values(['Date','Member Name'],ascending=True).reset_index(drop=True)


#Assign Trip Legs to the trips
alv_all['Leg'] = 'A'

# Assign 'A' to the 'Duplicate' column for non-duplicate rows
alv_all.loc[~alv_all.duplicated(['Member Name','Date']), 'Leg'] = 'A'

# Assign 'B' to the 'Duplicate' column for duplicate rows
alv_all.loc[alv_all.duplicated(['Member Name','Date'],'first'), 'Leg'] = 'B'


# Format the Name column
alv_all['First Name'] = alv_all['Member Name'].apply(lambda x : x.split(' ')[0])
alv_all['Last Name'] = alv_all['Member Name'].apply(lambda x : x.split(' ')[-1])

# Concatenate the 'First Name' and 'Last Name' columns with a comma in between
alv_all['Name'] = alv_all['First Name'] + ', ' + alv_all['Last Name']

# Drop the 'First Name' and 'Last Name' columns
alv_all = alv_all.drop(columns=['First Name', 'Last Name','Member Name'])

# Re arrange columns
alv_all = alv_all.loc[:,['Date','Name','Cost','Leg']]
alv_all['Trip'] = alv_all['Date'] + ' ' + alv_all['Name'] + ' ' + alv_all['Leg']

print('Alivi EOPS uploaded!!')
#alv_all.head(2)

Alivi EOPS uploaded!!


In [4]:
### Upload Modivcare EOPS

df = dd.read_csv('C:\\Users\\vjron\\OneDrive\\Desktop\\data science\\Rubicund Finance\\Modivcare\\LGTC*.csv',dtype={'Cancellation Reason': 'object','Manually Denied Comment': 'object','Manually Denied Reason': 'object','Manually Denied Trip ID': 'object'}).compute()
df = pd.DataFrame(df)

# Filter only needed columns
df = df.loc[:,['Trip ID','Trip Leg','Trip Leg Date','Rider','LCI Miles','Amount','Invoice Number','Batch','Record Type','Denial Note','Manually Denied Comment']]
#df = df.loc[:,['Trip ID','Trip Leg','Trip Leg Date','Rider','LCI Miles','Amount']]

# Rename columns
mod_all = df.rename(columns={'Trip Leg':'Leg','Trip Leg Date':'Date','Rider':'Name','LCI Miles':'Miles','Amount':'Cost'})


# Define a function to remove the dollar sign from cost column
def remove_dollar_sign(s):
    return s.replace('$','')

# Apply the function to the 'amount' column
mod_all['Cost'] = mod_all['Cost'].apply(remove_dollar_sign)

# Change cost column to float
mod_all['Cost'] = mod_all['Cost'].astype(float)


# Sort data
mod_all.dropna().sort_values(['Date','Name'],ascending=True).reset_index(drop=True)

# Combine Date, Trip ID and Leg to form Authorization column
mod_all['Date'] = pd.to_datetime(mod_all['Date'], format='%m/%d/%y')
mod_all['Date'] = mod_all['Date'].dt.strftime('%Y/%m/%d') #Show the year column in yyyy format

mod_all['date_temp'] = mod_all['Date'].str.replace(r'\D', '')
mod_all['Trip ID'] = mod_all['Trip ID'].astype(str)

mod_all['Authorization'] = '1-' + mod_all['date_temp'] + '-' + mod_all['Trip ID'] + '-' + mod_all['Leg']
mod_all = mod_all.drop('date_temp', axis=1)


# Get all the rows that contain verified paid**  under the record type column  
mod_all = mod_all.loc[mod_all['Record Type'].str.contains('Verified-Paid')]    

# Get only the authorization column
mod_all = mod_all.loc[:,['Authorization']]


#mod_all.to_csv('C:\\Users\\vjron\\OneDrive\\Desktop\\data science\\Rubicund Finance\\Modivcare checks.csv',index=False)

print('Modivcare EOPS uploaded!!')
#mod_all

Modivcare EOPS uploaded!!


In [5]:
#### Check Alivi Invoices

# Filter Alivi trips
alv = dt[dt['Funding Source'] == 'Alivi']
alv = alv.loc[:,['Date','Name','Cost']]


#Assign Trip Legs to the trips
alv['Leg'] = 'A'

# Assign 'A' to the 'Duplicate' column for non-duplicate rows
alv.loc[~alv.duplicated(['Name','Date']), 'Leg'] = 'A'

# Assign 'B' to the 'Duplicate' column for duplicate rows
alv.loc[alv.duplicated(['Name','Date'],'first'), 'Leg'] = 'B'

alv['Trip'] = alv['Date'] + ' ' + alv['Name'] + ' ' + alv['Leg']

#print(alv_all['Name']=='WILLIAMS RICHARD')


# Get payments not found on EOP
#alivi = pd.merge(alv_all,alv, how='inner').drop_duplicates('Reference Number')
#alivi = alivi.loc[:,['Reference Number','Date','Leg','Name','Miles','Cost']]


alivi = pd.merge(alv_all,alv, how='outer', indicator=True)
alivi = alivi[alivi['_merge']=='right_only']
#alivi = alivi.drop('_merge',axis=1)

# Get payments not found on EOP 
#modiv = pd.merge(mod_all,mod, on='Authorization', how='outer', indicator=True)
#modiv = modiv[modiv['_merge']=='right_only']
#modiv = modiv.drop('_merge', axis=1)




# Show sum of unpaid invoices and details
#print('Total Alivi Unpaid Invoice = ', alivi['Cost'].sum())

#alivi.to_csv('C:\\Users\\vjron\\OneDrive\\Desktop\\data science\\Rubicund Finance\\Alivi Pending Invoices.csv',index=False)

#alivi

In [10]:
#### Check Modivcare data from Mediroutes

# Filter Alivi trips
mod = dt[dt['Funding Source'] == 'Modivcare']
mod = mod.loc[:,['Date','Name','Cost','Authorization']]

#modiv = mod[mod['Authorization'].isin(mod_all['Authorization'])]

# Get payments not found on EOP 
modiv = pd.merge(mod_all,mod, on='Authorization', how='outer', indicator=True)
modiv = modiv[modiv['_merge']=='right_only']
modiv = modiv.drop('_merge', axis=1)


# Show sum of unpaid invoices and details
print('Total Modivcare Unpaid Invoice = ', modiv['Cost'].sum())

#modiv.to_csv('C:\\Users\\vjron\\OneDrive\\Desktop\\data science\\Rubicund Finance\\Modivcare Pending Invoices.csv',index=False)

#modiv

Total Modivcare Unpaid Invoice =  204284.29


In [20]:
gc.collect() # Clear memory

#### Drivers Pay
# Create a NumPy array containing Runs and Daily rates
rate = np.array([['Beulah',154.28, 'Beulah Minter', '10/28/2022'], 
                 ['Erica',160, 'Erica Williams', '08/04/2022'],
                 ['Cedric',160, 'Cedric', '01/18/2023'],
                 ['BENEDICT',150.28, 'Benedict Kamara', '01/10/2023'],
                 ['Valeria',154.28, 'Valeria DeOliveria', '11/05/2022'],
                 ['Jenell',154.28, 'Jenell Jones Johnson', '12/13/2022'],
                 ['BAYO',154.28, 'Adebayo Olasupo', '12/13/2022'],
                 ['FRED',154.28, 'Fredison Kpogban', '04/17/2023'],
                 ['Anthony',154.28, 'Anthony Sseremba', '05/05/2023'],
                 ['Jamil',154.28, 'Jamil Adebello', '08/13/2023'],
                 ['SAMARIA',154.28, 'Brittani Castle', '08/21/2023'],
                 ['OPE',154.28, 'Opeoluwa Caston', '10/28/2023'],
                 ['BONA',154.28, 'Bona Abraham', '12/13/2023'],
                 ['WALE',160, 'Wale', '12/13/2023'],
                 ['Yousuf',180, 'Amir Synders', '07/24/2023']])

rates = pd.DataFrame(rate, columns=['Run', 'rate','Driver','Start Date'])     # Create a DataFrame from the array
rates['rate'].astype(float)


# Get data from Mediroutes and clean the data
run = dt.loc[:,['Date','Run']]
run['Date'] = pd.to_datetime(run['Date'])

# Select the date you want to process
date_from = '2024-03-10'   #yyyy-mm-dd
date_to = '2024-03-23'
check_date = '29-03-2024'  #mm-dd-yyyy

#date_from = '2023-06-04'   #yyyy-mm-dd
#date_to = '2023-06-17'
#check_date = '03-03-2023'  #mm-dd-yyyy

run = run.loc[run['Date'].between(date_from, date_to)].sort_values('Run', ascending=True).drop_duplicates()

#Get the number of days worked per run 
run1 = run.groupby(['Run']).count()   # number of days worked


# Merge the run and daily rates data
run_test = pd.merge(run1,rates, on='Run', how='inner').drop_duplicates()

run_test['Net Pay'] = run_test['Date']*run_test['rate'].astype(float)
run_test['Net Pay'] = run_test['Net Pay'].round(2)


run_test = run_test.rename(columns={'Date':'Days worked'})
run_test = run_test.loc[:,['Run','Start Date','Driver','Days worked','rate','Net Pay']]

print('Time frame ', date_from, 'to', date_to)
print('Total pay = ', run_test['Net Pay'].sum())
run_test


Time frame  2024-03-10 to 2024-03-23
Total pay =  4674.28


,Run,Start Date,Driver,Days worked,rate,Net Pay
0,Cedric,01/18/2023,Cedric,9,160,1440.00
1,OPE,10/28/2023,Opeoluwa Caston,1,154.28,154.28
2,WALE,12/13/2023,Wale,8,160,1280.00
3,Yousuf,07/24/2023,Amir Synders,10,180,1800.00


In [21]:
driver_name = 'Wale'

run_tt = pd.merge(run,rates, on='Run', how='inner').drop_duplicates().sort_values('Date', ascending=True)
run_tt = run_tt.loc[:,['Driver','Date','rate']]

print('                                              EXPLANATION OF PAYMENT                             ')
print('')
print('')
print('Duration:', date_from, '-', date_to)
print()
print('CONTRACTOR NAME: ', driver_name)
print('SSN: XXX-XX-XXXX')
print('SERVICE PROVIDED: NEMT DRIVER SERVICES')
print()
#print('START DATE: ', run_tt.loc[run_tt['Start Date'] == run_name].to_string(index=False))

print('DEDUCTIONS')
print('GL: $9.72')
print('Tax withheld: $0.00')
print()
print()
print('DAILY EARNINGS')
print('Rate: $120.00')
print('Gas: $40.00')
#print('Toll: $4.00')
print()
print()
print(run_test.loc[run_test['Driver'] == driver_name].to_string(index=False))
print()
print('Check date: ',check_date)
print()
print('schedule grid')
print(run_tt.loc[run_tt['Driver'] == driver_name].to_string(index=False))

                                              EXPLANATION OF PAYMENT                             


Duration: 2024-03-10 - 2024-03-23

CONTRACTOR NAME:  Wale
SSN: XXX-XX-XXXX
SERVICE PROVIDED: NEMT DRIVER SERVICES

DEDUCTIONS
GL: $9.72
Tax withheld: $0.00


DAILY EARNINGS
Rate: $120.00
Gas: $40.00


  Run  Start Date Driver  Days worked rate  Net Pay
 WALE  12/13/2023   Wale            8  160   1280.0

Check date:  29-03-2024

schedule grid
Driver       Date rate
  Wale 2024-03-11  160
  Wale 2024-03-12  160
  Wale 2024-03-13  160
  Wale 2024-03-18  160
  Wale 2024-03-19  160
  Wale 2024-03-20  160
  Wale 2024-03-21  160
  Wale 2024-03-22  160


In [22]:
# bayo - 2nd, 3rd, - pay = 1079.96
# antho - pay = 771.40
#   Jamil - 29th pay $77.14; pay = 539.98
# Jenell - 29th pay $0 ; pay = 1079.96
#total = 3471.30

#bayo - $694.26 - half day on 23rd oct 
